In [4]:
%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from astropy.io import fits
import astropy.units as u
from astroquery.simbad import Simbad

In [58]:
Simbad.get_field_description('sptype')

all fields related with the spectral type


In [8]:
tic_gaia_table = pd.read_csv('ticGAIA1to13.csv')
df = tic_gaia_table.drop_duplicates('TIC')
len(tic_gaia_table)

118939

In [9]:
customSimbad = Simbad()
customSimbad.TIMEOUT = 60*60
customSimbad.add_votable_fields('otype')

query_names = []
for sid in df['source_id'].values:
    query_names.append('Gaia DR2 ' + str(sid))

In [25]:
# Cant query all objects at once
result = customSimbad.query_objects(query_names[100001:])

/astro/users/scw7/anaconda3/lib/python3.7/site-packages/astroquery/simbad/core.py:136: UserWarning: Warning: The script line number 3 raised an error (recorded in the `errors` attribute of the result table): Identifier not found in the database : Gaia DR2 6111777533533213184 
  (error.line, error.msg))
/astro/users/scw7/anaconda3/lib/python3.7/site-packages/astroquery/simbad/core.py:136: UserWarning: Warning: The script line number 30 raised an error (recorded in the `errors` attribute of the result table): Identifier not found in the database : Gaia DR2 6112257264201415552 
  (error.line, error.msg))
/astro/users/scw7/anaconda3/lib/python3.7/site-packages/astroquery/simbad/core.py:136: UserWarning: Warning: The script line number 107 raised an error (recorded in the `errors` attribute of the result table): Identifier not found in the database : Gaia DR2 6113926872606416768 
  (error.line, error.msg))
/astro/users/scw7/anaconda3/lib/python3.7/site-packages/astroquery/simbad/core.py:136

In [26]:
result3 = result

In [24]:
result2 = result

In [22]:
result1 = result

In [32]:
from astropy.table import vstack
result_full = vstack([result1, result2, result3])

In [34]:
# result_full.write('1to13_simbad.txt', format='ascii')

In [5]:
from astropy.table import Table
t = Table.read('1to13_simbad.txt', format='ascii')

In [6]:
otypes = np.array(t['OTYPE'])
flare_row_ind = np.where(otypes == "b'Flare*'")
t[flare_row_ind]

MAIN_ID,RA,DEC,RA_PREC,DEC_PREC,COO_ERR_MAJA,COO_ERR_MINA,COO_ERR_ANGLE,COO_QUAL,COO_WAVELENGTH,COO_BIBCODE,OTYPE
str36,str13,str13,int64,int64,float64,float64,int64,str1,str1,str22,str22
b'2XMM J040807.0-712700',04 08 07.0991,-71 27 00.853,14,14,0.022,0.022,90,A,O,b'2018yCat.1345....0G',b'Flare*'
b'RBS 35',00 15 02.3400,-72 50 32.846,14,14,0.352,0.363,90,A,O,b'2018yCat.1345....0G',b'Flare*'
b'GR* 4',00 15 36.7132,-29 46 00.543,14,14,0.081,0.058,90,A,O,b'2018yCat.1345....0G',b'Flare*'
b'EXO 0235.2-5216',02 36 51.7015,-52 03 03.601,14,14,0.028,0.03,90,A,O,b'2018yCat.1345....0G',b'Flare*'
b'CD-56 1032A',04 53 31.2015,-55 51 37.104,14,14,0.029,0.035,90,A,O,b'2018yCat.1345....0G',b'Flare*'
b'CD-56 1032B',04 53 30.5452,-55 51 31.665,14,14,0.036,0.044,90,A,O,b'2018yCat.1345....0G',b'Flare*'
b'GJ 1033',01 13 24.0253,-22 54 07.752,14,14,0.046,0.041,90,A,O,b'2018yCat.1345....0G',b'Flare*'
b'G 268-131',01 09 18.7263,-24 30 23.682,14,14,0.055,0.043,90,A,O,b'2018yCat.1345....0G',b'Flare*'
b'G 73-45',02 20 46.2532,+02 58 37.582,14,14,0.108,0.073,90,A,O,b'2018yCat.1345....0G',b'Flare*'


In [90]:
from astroquery.mast import Observations

flare_star_names = np.asarray(t[flare_row_ind]['MAIN_ID'])
SIMBAD_flare_TICs = []
for idx, val in enumerate(flare_star_names):
    name = val.split('\'')[1]
    obs_table = Observations.query_object(name)
    tess_row = obs_table[obs_table['obs_collection'] == 'TESS']
    tess_row = tess_row[tess_row['target_name'] != 'TESS FFI']
    TIC = tess_row[0]['target_name']
    SIMBAD_flare_TICs.append(TIC)

In [91]:
SIMBAD_flare_TICs

['25132999',
 '266998480',
 '12509218',
 '166808151',
 '220433363',
 '220433364',
 '268717977',
 '326198086',
 '420001446',
 '93032256',
 '400006453',
 '146522418',
 '169344643',
 '49934962',
 '415508270',
 '282501711',
 '323688555']

In [100]:
df_flare = pd.read_csv('1to13_filt_flare_out.csv')
unique_flare_tics = np.unique(df_flare['TIC'].values).astype(int)
match_mask = np.isin(unique_flare_tics, SIMBAD_flare_TICs)
len(unique_flare_tics[match_mask == True])

0

In [ ]:
# None of our flare detections were previously known flare stars? Really?